In [2]:
import pandas as pd
import numpy as np
import os
import math

In [3]:
df = pd.read_csv('nfl.csv', low_memory=False)

In [4]:
df = df.sort_values(['Date', 'GameID', 'TimeSecs'], ascending=False)
df = df[0 <= df.TimeSecs]
df.index = range(df.shape[0])

In [5]:
df.to_csv('nfl_refined.csv')

In [116]:
#add fumbles, sacks, penalty yardage
gameid=[]; hometm=[]; awaytm=[]; homesc=[]; awaysc=[]; hmhalfsc=[]; awhalfsc=[]; ot=[]
hpassyds=[0]; hpatts=[0]; hypa=[]; hcomp=[0]; hcpct=[]; hints=[0]; hrushyds=[0]; hratts=[0]; hypr=[]; hsacks=[0]; hsackyds=[0]; hfums=[0]; hfumlost=[0];
apassyds=[0]; apatts=[0]; aypa=[]; acomp=[0]; acpct=[]; aints=[0]; arushyds=[0]; aratts=[0]; aypr=[]; asacks=[0]; asackyds=[0]; afums=[0]; afumlost=[0]
hmadj=0; awadj=0
for i in range(5000):
    if df.qtr[i] == 4 and not df.qtr[i+1] == 4:
        if df.qtr[i+1] == 1:
            ot.extend([0])
        else:
            ot.extend([1])
        offscore = df.PosTeamScore[i-1] #score after 4 quarters
        defscore = df.DefTeamScore[i-1]
        fgpts = 0
        if math.isnan(offscore):
            offscore = df.PosTeamScore[i]
            defscore = df.DefTeamScore[i]
            if df.FieldGoalResult[i] == 'Good':
                fgpts += 3
        if df.posteam[i] == df.HomeTeam[i]:
            if df.Touchdown[i] == 1 and not df.sp[i] == 1:
                hmadj += 6
                if df.ExPointResult[i] == 'Good': hmadj += 1
                elif df.TwoPointConv[i] == 'Success': hmadj += 2
            elif df.Touchdown[i] == 1 and df.sp[i] == 1:
                awadj += 6
                if df.ExPointResult[i] == 'Good': awadj += 1
                elif df.TwoPointConv[i] == 'Success': awadj += 2
            homesc.extend([offscore+hmadj+fgpts])
            awaysc.extend([defscore+awadj])
        else:
            if df.Touchdown[i] == 1 and not df.sp[i] == 1:
                awadj += 6
                if df.ExPointResult[i] == 'Good': awadj += 1
                elif df.TwoPointConv[i] == 'Success': awadj += 2
            elif df.Touchdown[i] == 1 and df.sp[i] == 1:
                hmadj += 6
                if df.ExPointResult[i] == 'Good': hmadj += 1
                elif df.TwoPointConv[i] == 'Success': hmadj += 2
            homesc.extend([defscore+hmadj])
            awaysc.extend([offscore+awadj+fgpts])
        hmadj=0; awadj=0 # will be uncommented
    if df.qtr[i] == 4 and not df.qtr[i+1] == 4: #4 to be changed to 2
        gameid.extend([df.GameID[i]])
        hometm.extend([df.HomeTeam[i]])
        awaytm.extend([df.AwayTeam[i]])
        hypa.extend([hpassyds[-1]/hpatts[-1]]); hcpct.extend([hcomp[-1]/hpatts[-1]])
        hypr.extend([hrushyds[-1]/hratts[-1]])
        hpassyds.append(0); hpatts.append(0); hcomp.append(0); hints.append(0); hrushyds.append(0); hratts.append(0)
        hsacks.append(0); hsackyds.append(0); hfums.append(0); hfumlost.append(0)
        aypa.extend([apassyds[-1]/apatts[-1]]); acpct.extend([acomp[-1]/apatts[-1]])
        aypr.extend([arushyds[-1]/aratts[-1]])
        apassyds.append(0); apatts.append(0); acomp.append(0); aints.append(0); arushyds.append(0); aratts.append(0)
        asacks.append(0); asackyds.append(0); afums.append(0); afumlost.append(0)
        
        offscore = df.PosTeamScore[i+1] #halftime score 
        defscore = df.DefTeamScore[i+1]
        if df.posteam[i+1] == df.HomeTeam[i]:
            hmhalfsc.extend([offscore+hmadj])
            awhalfsc.extend([defscore+awadj])
        else:
            hmhalfsc.extend([defscore+hmadj])
            awhalfsc.extend([offscore+awadj])
        #hmadj=0; awadj=0 #should be commented out 
    if df.qtr[i] in [1,2,3,4]: #need all 4 quarters for full time score
        if df.Touchdown[i] == 1:
            j=1
            while not isinstance(df.posteam[i+j], str):
                j+=1
            if not isinstance(df.ExPointResult[i+j], str) and not isinstance(df.TwoPointConv[i+j], str) and df['Challenge.Replay'][i] == 0:
                if df.HomeTeam[i] == df.DefensiveTeam[i]: 
                    hmadj += 1
                    if df.ScoreDiff[i] == df.ScoreDiff[i+j]:
                        hmadj += 6
                else:
                    awadj += 1
                    if df.ScoreDiff[i] == df.ScoreDiff[i+j]:
                        awadj += 6
    if df.qtr[i] in [1,2,3,4]: #to be changed to [1,2]
        if df.PassAttempt[i] == 1 and not df.PlayType[i] == 'No Play' and not isinstance(df.TwoPointConv[i], str):
            if df.posteam[i] == df.HomeTeam[i]:
                hpatts[-1] += 1
                if df.InterceptionThrown[i] == 1:
                    hints[-1] += 1
                elif df.PassOutcome[i] == 'Complete':
                    hpassyds[-1] += df.AirYards[i] + df.YardsAfterCatch[i]
                    hcomp[-1] += 1
#                 elif df['Yards.Gained'][i] >0 and df['Challenge.Replay'][i] == 1:
#                     hpassyds[-1] += df.AirYards[i] + df.YardsAfterCatch[i]
#                     hcomp[-1] += 1
            elif df.posteam[i] == df.AwayTeam[i]:
                apatts[-1] += 1
                if df.InterceptionThrown[i] == 1:
                    aints[-1] += 1
                elif df.PassOutcome[i] == 'Complete':
                    apassyds[-1] += df.AirYards[i] + df.YardsAfterCatch[i]
                    acomp[-1] += 1
        elif df.RushAttempt[i] == 1 and not df.PlayType[i] == 'No Play' and not isinstance(df.TwoPointConv[i], str):
            if df.posteam[i] == df.HomeTeam[i]:
                hratts[-1] += 1
                if df['Challenge.Replay'][i] == 1:
                    hrushyds[-1] += (df.yrdline100[i]-df.yrdline100[i+1])
                else:
                    hrushyds[-1] += df['Yards.Gained'][i]
            elif df.posteam[i] == df.AwayTeam[i]:
                aratts[-1] += 1
                if df['Challenge.Replay'][i] == 1:
                    arushyds[-1] += (df.yrdline100[i]-df.yrdline100[i+1])
                else:
                    arushyds[-1] += df['Yards.Gained'][i]
        elif df.Sack[i] == 1:
            if df.posteam[i] == df.HomeTeam[i]:
                hsacks[-1] += 1
                hsackyds[-1] += df['Yards.Gained'][i]
            elif df.posteam[i] == df.AwayTeam[i]:
                asacks[-1] += 1
                asackyds[-1] += df['Yards.Gained'][i]
        if df.Fumble[i] == 1:
            if df.posteam[i] == df.HomeTeam[i]:
                hfums[-1] += 1
                if df.RecFumbTeam[i] == df.AwayTeam[i]:
                    hfumlost[-1] += 1
            elif df.posteam[i] == df.AwayTeam[i]:
                afums[-1] += 1
                if df.RecFumbTeam[i] == df.HomeTeam[i]:
                    afumlost[-1] += 1
                

In [117]:
ot.extend([0]); homesc.extend([0]); awaysc.extend([0])

hpassyds = hpassyds[:-1]; hpatts = hpatts[:-1]; hcomp = hcomp[:-1]; hints = hints[:-1]; hrushyds = hrushyds[:-1]
hratts = hratts[:-1]; hsacks = hsacks[:-1]; hsackyds = hsackyds[:-1]; hfums = hfums[:-1]; hfumlost = hfumlost[:-1]
apassyds = apassyds[:-1]; apatts = apatts[:-1]; acomp = acomp[:-1]; aints = aints[:-1]; arushyds = arushyds[:-1]
aratts = aratts[:-1]; asacks = asacks[:-1]; asackyds = asackyds[:-1]; afums = afums[:-1]; afumlost = afumlost[:-1]

In [118]:
wrangled = pd.DataFrame({'gameid':gameid, 'home':hometm, 'away':awaytm, 'homesc':homesc, 'awaysc':awaysc, 
                         'hmhalfsc':hmhalfsc, 'awhalfsc':awhalfsc, 'ot':ot,
                         'hpyd':hpassyds, 'hpatt':hpatts, 'hcomp':hcomp, 'hypa':hypa, 'hcomppct':hcpct, 'hint':hints,
                         'hryd':hrushyds, 'hratt':hratts, 'hypr':hypr, 'hsacks':hsacks, 'hsackyds':hsackyds,
                         'hfum':hfums, 'hfuml':hfumlost,
                         'apyd':apassyds, 'apatt':apatts, 'acomp':acomp, 'aypa':aypa, 'acomppct':acpct, 'aint':aints,  
                         'aryd':arushyds, 'aratt':aratts, 'aypr':aypr, 'asacks':asacks, 'asackyds':asackyds,
                        'afum':afums, 'afuml':afumlost})

In [119]:
htotyds = wrangled[['hpyd','hryd','hsackyds']].transpose().sum()
atotyds = wrangled[['apyd','aryd','asackyds']].transpose().sum()
htos = wrangled[['hint','hfuml']].transpose().sum()
atos = wrangled[['aint','afuml']].transpose().sum()
wrangled['htotyds'] = htotyds; wrangled['atotyds'] = atotyds
wrangled['htos'] = htos; wrangled['atos'] = atos

In [120]:
wrangled = wrangled.set_index('gameid')

In [121]:
wrangled.head(28).loc[:,['home','away','homesc','awaysc',
                  'hpyd','hpatt','hcomp','hint','hryd','hratt','hsacks','hsackyds','htotyds','hfum','hfuml','htos',
                  'apyd','apatt','acomp','aint','aryd','aratt','asacks','asackyds','atotyds','afum','afuml','atos']]

,home,away,homesc,awaysc,hpyd,hpatt,hcomp,hint,hryd,hratt,...,acomp,aint,aryd,aratt,asacks,asackyds,atotyds,afum,afuml,atos
gameid,,,,,,,,,,,,,,,,,,,,,
2017123115,SEA,ARI,24.0,26.0,221,29,18,0,99.0,22,...,15,1,117.0,32,1,-7,255.0,1,0,1
2017123114,LA,SF,13.0,34.0,169,34,20,0,102.0,26,...,20,2,172.0,28,0,0,464.0,2,1,3
2017123113,LAC,OAK,30.0,10.0,387,37,28,0,119.0,28,...,19,1,106.0,23,2,-12,337.0,1,1,2
2017123112,DEN,KC,24.0,27.0,254,37,21,2,107.0,25,...,22,1,149.0,26,2,-15,418.0,2,1,2
2017123111,TEN,JAX,15.0,10.0,134,21,12,0,184.0,38,...,15,2,82.0,24,2,-12,228.0,2,1,3
2017123110,TB,NO,30.0,31.0,363,51,28,3,110.0,28,...,22,0,92.0,26,2,-14,323.0,0,0,0
2017123109,PIT,CLE,28.0,24.0,239,27,23,1,130.0,28,...,16,1,102.0,27,6,-42,374.0,1,1,2
2017123108,PHI,DAL,0.0,6.0,167,33,22,1,70.0,18,...,17,0,129.0,32,1,-7,301.0,1,0,0
2017123107,NYG,WAS,18.0,10.0,138,29,11,1,264.0,41,...,20,3,61.0,16,3,-22,197.0,0,0,3


In [115]:
wrangled.to_csv('wrangled.csv')

In [16]:
list(df.columns)

['Date',
 'GameID',
 'Drive',
 'qtr',
 'down',
 'time',
 'TimeUnder',
 'TimeSecs',
 'PlayTimeDiff',
 'SideofField',
 'yrdln',
 'yrdline100',
 'ydstogo',
 'ydsnet',
 'GoalToGo',
 'FirstDown',
 'posteam',
 'DefensiveTeam',
 'desc',
 'PlayAttempted',
 'Yards.Gained',
 'sp',
 'Touchdown',
 'ExPointResult',
 'TwoPointConv',
 'DefTwoPoint',
 'Safety',
 'Onsidekick',
 'PuntResult',
 'PlayType',
 'Passer',
 'Passer_ID',
 'PassAttempt',
 'PassOutcome',
 'PassLength',
 'AirYards',
 'YardsAfterCatch',
 'QBHit',
 'PassLocation',
 'InterceptionThrown',
 'Interceptor',
 'Rusher',
 'Rusher_ID',
 'RushAttempt',
 'RunLocation',
 'RunGap',
 'Receiver',
 'Receiver_ID',
 'Reception',
 'ReturnResult',
 'Returner',
 'BlockingPlayer',
 'Tackler1',
 'Tackler2',
 'FieldGoalResult',
 'FieldGoalDistance',
 'Fumble',
 'RecFumbTeam',
 'RecFumbPlayer',
 'Sack',
 'Challenge.Replay',
 'ChalReplayResult',
 'Accepted.Penalty',
 'PenalizedTeam',
 'PenaltyType',
 'PenalizedPlayer',
 'Penalty.Yards',
 'PosTeamScore',
 'De

In [70]:
#Tyler's part
nfl = pd.read_csv('nfl.csv', low_memory=False)
spread = pd.read_csv('nflspreads.csv')

teams = {"ARI": "Arizona Cardinals", "ATL": "Atlanta Falcons", "BAL": "Baltimore Ravens",
         "BUF": "Buffalo Bills", "CAR": "Carolina Panthers", "CHI": "Chicago Bears",
         "CIN": "Cincinnati Bengals", "CLE": "Cleveland Browns", "DAL": "Dallas Cowboys",
         "DEN": "Denver Broncos", "DET": "Detroit Lions", "GB": "Green Bay Packers",
         "HOU": "Houston Texans", "IND": "Indianapolis Colts", "JAX": "Jacksonville Jaguars",
         "JAC": "Jacksonville Jaguars", "KC": "Kansas City Chiefs", "LA": "Los Angeles Rams",
         "LAC": "Los Angeles Chargers", "MIA": "Miami Dolphins", "MIN": "Minnesota Vikings",
         "NE": "New England Patriots", "NO": "New Orleans Saints", "NYG": "New York Giants",
         "NYJ": "New York Jets", "OAK": "Oakland Raiders", "PHI": "Philadelphia Eagles",
         "PIT": "Pittsburgh Steelers", "SD": "San Diego Chargers", "SEA": "Seattle Seahawks",
         "SF": "San Francisco 49ers", "STL": "St. Louis Cardinals", "TB": "Tampa Bay Buccaneers",
         "TEN": "Tennessee Titans", "WAS": "Washington Redskins"}

"""Get desired features to include"""
nfl_teams = nfl[["Date", "GameID", "HomeTeam", "AwayTeam"]]
spread_games = list()
for game in range(len(spread["spread_favorite"])):
    if not math.isnan(spread["spread_favorite"][game]):
        if int(spread["schedule_date"][game][6:10]) > 2009:
            spread_games.append(spread.iloc[game])
    else:
        pass
spread_games = pd.DataFrame(spread_games)

"""Clean up data to prepare for merging"""
spread_dates_list = list()
for date in spread_games["schedule_date"]:
    spread_dates_list.append(date)
for date in range(len(spread_dates_list)):
    spread_dates_list[date] = spread_dates_list[date][6:10] + "/" + \
                              spread_dates_list[date][0:2] + "/" + \
                              spread_dates_list[date][3:5]
spread_games["schedule_date"] = spread_dates_list
nfl_dates_list = list()
for date in nfl_teams["Date"]:
    nfl_dates_list.append(date.replace("-", "/"))
nfl_teams.loc[:]["Date"] = nfl_dates_list
nfl_dates_set = set()
for date in nfl_dates_list:
    nfl_dates_set.add(date)
spread_dates = set(spread_dates_list)

"""Merge data sets"""
accounted_dates = nfl_dates_set.intersection(spread_dates)
nfl_df = pd.DataFrame(list(accounted_dates), columns=["Date"])
nfl_teams = np.array(nfl_teams)
unique_games = set()
for game in nfl_teams:
    unique_games.add(tuple(game))
matched_games = set()
for game in unique_games:
    if game[0] in accounted_dates:
        matched_games.add(game)
accounted_spread_games = spread_games[["schedule_date", "team_home", "team_away",
                                       "score_home", "score_away", "team_favorite_id",
                                       "spread_favorite", "over_under_line"]]
accounted_spread_array = np.array(accounted_spread_games)
matched_scores = list()
for game in accounted_spread_array:
    for g in matched_games:
        if game[0] == g[0] and game[1] == teams[g[2]] and game[2] == teams[g[3]]:
            game = tuple(game) + (g[1],)
            matched_scores.append(tuple(game))
new_df = pd.DataFrame(matched_scores, columns=["Date", "Home Team", "Away Team",
                                               "Home Score", "Away Score", "Favorite Team ID",
                                               "Spread Favorite", "Over Under", "gameid"])
new_df = new_df.set_index('gameid')

/Users/eddieowens/miniconda3/envs/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/eddieowens/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [73]:
merged = new_df.merge(wrangled, how='outer', on='gameid')

In [76]:
merged.to_csv('nfl_cleaned.csv')

In [77]:
merged.head()

,Date,Home Team,Away Team,Home Score,Away Score,Favorite Team ID,Spread Favorite,Over Under,home,away,...,hypr,apyd,apatt,acomp,aypa,acomppct,aint,aryd,aratt,aypr
gameid,,,,,,,,,,,,,,,,,,,,,
2010010311,2010/01/03,Arizona Cardinals,Green Bay Packers,7.0,33.0,ARI,-3.0,44,ARI,GB,...,2.000000,153,16,12,9.562500,0.750000,0,64.0,17,3.764706
2010010300,2010/01/03,Buffalo Bills,Indianapolis Colts,30.0,7.0,BUF,-8.0,32,BUF,IND,...,5.684211,116,23,15,5.043478,0.652174,2,-4.0,7,-0.571429
2010010301,2010/01/03,Carolina Panthers,New Orleans Saints,23.0,10.0,CAR,-10.0,41,CAR,NO,...,5.833333,32,11,7,2.909091,0.636364,0,54.0,15,3.600000
2010010302,2010/01/03,Cleveland Browns,Jacksonville Jaguars,23.0,17.0,CLE,-2.0,35,CLE,JAC,...,4.956522,68,13,9,5.230769,0.692308,1,48.0,14,3.428571
2010010303,2010/01/03,Dallas Cowboys,Philadelphia Eagles,24.0,0.0,DAL,-3.0,47,DAL,PHI,...,6.933333,144,17,12,8.470588,0.705882,0,12.0,5,2.400000
